This model will require ffmpeg package installation in the conda-env, system wide installation is not required.

Use 'pip install python-ffmpeg' wrapper package for installation.

Right now runs in the stable_env environment.

In [1]:
import os 
print(os.getenv("CONDA_DEFAULT_ENV"))

stable_env


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoModel, AutoImageProcessor

In [3]:
model_name = "DAMO-NLP-SG/VideoLLaMA3-7B"

### Load Model

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# model.config

In [19]:
model.model

Videollama3Qwen2Model(
  (embed_tokens): Embedding(152064, 3584)
  (layers): ModuleList(
    (0-27): 28 x Qwen2DecoderLayer(
      (self_attn): Qwen2Attention(
        (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
        (k_proj): Linear(in_features=3584, out_features=512, bias=True)
        (v_proj): Linear(in_features=3584, out_features=512, bias=True)
        (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
        (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
        (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((3584,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
  (vision_encoder): Videol

### Set Question & Video Path

In [7]:
video_path = '/home/aritrad/MSR-Project/random/insertion_sort.mp4'
question =  'How many element is present in the array shown?'

### Create Chat

In [8]:
# Video conversation
conversation = [
    {
        "role": "system",
        "content": (
            "You are a vision-language model analyzing lecture videos. "
            "You must rely strictly on visual evidence from the video frames.\n\n"
            "Follow these steps:\n"
            "1. Carefully examine the video frames.\n"
            "2. Perform OCR on any readable text appearing on slides, blackboards, or written material.\n"
            "3. Explicitly list the extracted text before reasoning.\n"
            "4. Answer the question using ONLY the extracted visual information.\n"
            "5. If the required information is not visible or readable, say "
            "\"The answer cannot be determined from the video.\" Do NOT guess.\n"
        )
    },
    {
        "role": "user",
        "content": [
            {
                "type": "video", 
                "video": 
                {
                    "video_path": video_path, 
                    "fps": 1, 
                    "max_frames": 240
                }
            },
            {
                "type": "text", "text": question
            },
        ]
    },
]

In [9]:
inputs = processor(conversation=conversation, return_tensors="pt")
inputs = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

In [10]:
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)

## Generate

In [11]:
output_ids = model.generate(
    **inputs, 
    max_new_tokens=512, 
    do_sample=False
)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The array has 10 elements.
